In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [5]:
# twitter_df.index

In [6]:
#################### TIMEINDEX FUNCTIONS #####################
def get_day_window_size_from_freq(dataset):#, freq='CBH'):
    
    if dataset.index.freq == custom_BH_freq():
        return 7
    
    if dataset.index.freq=='T':
        day_window_size = 1440
    elif dataset.index.freq=='BH':
        day_window_size = 8
    elif dataset.index.freq=='CBH':
        day_window_size = 7
    elif dataset.index.freq=='B':
        day_window_size=1
    elif dataset.index.freq=='D':
        day_window_size=1
        
    else:
        raise Exception('dataset freq=None')
        
    return day_window_size
    

def custom_BH_freq():
    import pandas as pd
    CBH = pd.tseries.offsets.CustomBusinessHour(start='09:30',end='16:30')
    return CBH
    
    
def  set_timeindex_freq(ive_df, col_to_fill=None, freq='CBH',fill_method='ffill',
                        verbose=3): #set_tz=True,
    
    import pandas as pd
    import numpy as np
    
    
    if verbose>1:
        # print(f"{'Index When:':>{10}}\t{'Freq:':>{20}}\t{'Index Start:':>{40}}\t{'Index End:':>{40}}")
        print(f"{'Index When:'}\t{'Freq:'}\t{'Index Start'}\t\t{'Index End:'}")
        print(f"Pre-Change\t{ive_df.index.freq}\t{ive_df.index[0]}\t{ive_df.index[-1]}")
        
    
    if freq=='CBH':
        freq=custom_BH_freq()
#         start_idx = 
        
    # Change frequency to freq
    ive_df = ive_df.asfreq(freq,)#'min')
    
    #     # Set timezone
    #     if set_tz==True:
    #         ive_df.tz_localize()
    #         ive_df.index = ive_df.index.tz_convert('America/New_York')
    
    # Report Success / Details
    if verbose>1:
        print(f"Post-Change\t{ive_df.index.freq}\t{ive_df.index[0]}\t{ive_df.index[-1]}")


    ## FILL AND TRACK TIMEPOINTS WITH MISSING DATA    
    
    # Helper Function for adding column to track the datapoints that were filled
    def check_null_times(x):
        import numpy as np
        if np.isnan(x):
            return True
        else:
            return False

    ## CREATE A COLUMN TO TRACK ROWS TO BE FILLED
    # If col_to_fill provided, use that column to create/judge ive_df['filled_timebin'] 
    if col_to_fill!=None:
        ive_df['filled_timebin'] = ive_df[col_to_fill].apply(lambda x: check_null_times(x))#True if ive_df.isna().any()
        
    # if not provided, use all columns and sum results
    elif col_to_fill == None:
        # Prefill fol with 0's
        ive_df['filled_timebin']=0
        
        # loop through all columns and add results of check_null_times from each loop
    for col in ive_df.columns:
        if ive_df[col].dtypes=='float64':
            #ive_df['filled_timebin'] = ive_df[target_col].apply(lambda x: check_null_times(x))#True if ive_df.isna().any()
            curr_filled_timebin_col = ive_df[col].apply(lambda x: check_null_times(x))#True if ive_df.isna().any() 

            # add results
            ive_df['filled_timebin'] +=  curr_filled_timebin_col
            
    ive_df['filled_timebin'] = ive_df['filled_timebin'] >0
            
    ## FILL IN NULL VALUES
    ive_df.fillna(method=fill_method, inplace=True)

    # Report # filled
    if verbose>0:
        check_fill = ive_df.loc[ive_df['filled_timebin']>0]
        print(f'\nFilled {len(check_fill==True)}# of rows using method {fill_method}')
    
    # Report any remaning null values
    if verbose>0:
        res = ive_df.isna().sum()
        if res.any():
            print(f'Cols with Nulls:')
            print(res[res>0])
        else:
            print('No Remaining Null Values')   
            
    # display header
    if verbose>2:
        display(ive_df.head())
    
    return ive_df


# Helper Function for adding column to track the datapoints that were filled
def check_null_times(x):
    import numpy as np
    if np.isnan(x):
        return True
    else:
        return False

In [33]:
# index_to_bin = df.index #.round('T')
# def get_hour(x):
    
group_hours = df.groupby(pd.Grouper(key='datetime',freq='H',axis=0)).groups

df['hour'] = group_hours.values()


# df_sec = df.asfreq('S')
# index_to_bin = df.index
# index_to_bin
# df_hour

ValueError: Length of values does not match length of index

In [113]:

############### TIMESERIES TESTS AND VISUALS ###############

def plot_time_series(stocks_df, freq=None, fill_method='ffill',figsize=(12,4)):
    
    df = stocks_df.copy()
    df.fillna(method=fill_method, inplace=True)
    df.dropna(inplace=True)
    
    if (df.index.freq==None) & (freq == None):
        xlabels=f'Time'
    
    elif (df.index.freq==None) & (freq != None):
        df = df.asfreq(freq)
        df.fillna(method=fill_method, inplace=True)
        df.dropna(inplace=True)
        xlabels=f'Time - Frequency = {freq}'

    else:
        xlabels=f'Time - Frequency = {df.index.freq}'
        
    ylabels="Price"

    raw_plot = df.plot(figsize=figsize)
    raw_plot.set_title('Stock Bid Closing Price ')
    raw_plot.set_ylabel(ylabels)
    raw_plot.set_xlabel(xlabels)
    
    
#################### TIMEINDEX FUNCTIONS #####################
def get_day_window_size_from_freq(dataset):#, freq='CBH'):
    
    if dataset.index.freq == custom_BH_freq():
        return 7
    
    if dataset.index.freq=='T':
        day_window_size = 1440
    elif dataset.index.freq=='BH':
        day_window_size = 8
    elif dataset.index.freq=='CBH':
        day_window_size = 7
    elif dataset.index.freq=='B':
        day_window_size=1
    elif dataset.index.freq=='D':
        day_window_size=1
        
    else:
        raise Exception('dataset freq=None')
        
    return day_window_size
    

def custom_BH_freq():
    import pandas as pd
    CBH = pd.tseries.offsets.CustomBusinessHour(start='09:30',end='16:30')
    return CBH
    
    
def  set_timeindex_freq(ive_df, col_to_fill=None, freq='CBH',fill_method='ffill',
                        verbose=3): #set_tz=True,
    
    import pandas as pd
    import numpy as np
    
    
    if verbose>1:
        # print(f"{'Index When:':>{10}}\t{'Freq:':>{20}}\t{'Index Start:':>{40}}\t{'Index End:':>{40}}")
        print(f"{'Index When:'}\t{'Freq:'}\t{'Index Start'}\t\t{'Index End:'}")
        print(f"Pre-Change\t{ive_df.index.freq}\t{ive_df.index[0]}\t{ive_df.index[-1]}")
        
    
    if freq=='CBH':
        freq=custom_BH_freq()
#         start_idx = 
        
    # Change frequency to freq
    ive_df = ive_df.asfreq(freq,)#'min')
    
    #     # Set timezone
    #     if set_tz==True:
    #         ive_df.tz_localize()
    #         ive_df.index = ive_df.index.tz_convert('America/New_York')
    
    # Report Success / Details
    if verbose>1:
        print(f"Post-Change\t{ive_df.index.freq}\t{ive_df.index[0]}\t{ive_df.index[-1]}")


    ## FILL AND TRACK TIMEPOINTS WITH MISSING DATA    
    
    # Helper Function for adding column to track the datapoints that were filled
    def check_null_times(x):
        import numpy as np
        if np.isnan(x):
            return True
        else:
            return False

    ## CREATE A COLUMN TO TRACK ROWS TO BE FILLED
    # If col_to_fill provided, use that column to create/judge ive_df['filled_timebin'] 
    if col_to_fill!=None:
        ive_df['filled_timebin'] = ive_df[col_to_fill].apply(lambda x: check_null_times(x))#True if ive_df.isna().any()
        
    # if not provided, use all columns and sum results
    elif col_to_fill == None:
        # Prefill fol with 0's
        ive_df['filled_timebin']=0
        
        # loop through all columns and add results of check_null_times from each loop
    for col in ive_df.columns:
        if ive_df[col].dtypes=='float64':
            #ive_df['filled_timebin'] = ive_df[target_col].apply(lambda x: check_null_times(x))#True if ive_df.isna().any()
            curr_filled_timebin_col = ive_df[col].apply(lambda x: check_null_times(x))#True if ive_df.isna().any() 

            # add results
            ive_df['filled_timebin'] +=  curr_filled_timebin_col
            
    ive_df['filled_timebin'] = ive_df['filled_timebin'] >0
            
    ## FILL IN NULL VALUES
    ive_df.fillna(method=fill_method, inplace=True)

    # Report # filled
    if verbose>0:
        check_fill = ive_df.loc[ive_df['filled_timebin']>0]
        print(f'\nFilled {len(check_fill==True)}# of rows using method {fill_method}')
    
    # Report any remaning null values
    if verbose>0:
        res = ive_df.isna().sum()
        if res.any():
            print(f'Cols with Nulls:')
            print(res[res>0])
        else:
            print('No Remaining Null Values')   
            
    # display header
    if verbose>2:
        display(ive_df.head())
    
    return ive_df


# Helper Function for adding column to track the datapoints that were filled
def check_null_times(x):
    import numpy as np
    if np.isnan(x):
        return True
    else:
        return False

# LOADING STOCK DATA 

In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

stock_df = pd.read_csv('data/stock_df_with_tech_indicators.csv')#, index_col=0, parse_dates=True)
stock_df['date'] = pd.to_datetime(stock_df['Unnamed: 0'])
stock_df.drop('Unnamed: 0',axis=1, inplace=True)
stock_df.set_index('date',inplace=True, drop=False)
stock_df = set_timeindex_freq(stock_df,verbose=0)
stock_df = stock_df.iloc[:,11:]

# DIsply input stock data
display(stock_df.head().style.set_caption('Raw Data'))
# plot_time_series(stock_df['price'])

,price,ma7,ma21,26ema,12ema,MACD,20sd,upper_band,lower_band,ema,momentum,date
date,,,,,,,,,,,,
2017-02-20 15:30:00,99.64,99.0808,97.6303,98.1047,98.6017,0.496972,1.13258,99.8954,95.3651,99.64,92.64,2017-02-20 15:30:00
2017-02-21 09:30:00,99.88,99.1218,97.6551,98.1289,98.6327,0.503812,1.14007,99.9353,95.375,99.8,92.88,2017-02-21 09:30:00
2017-02-21 10:30:00,100.37,99.172,97.6848,98.1593,98.6748,0.515495,1.15543,99.9957,95.374,100.18,93.37,2017-02-21 10:30:00
2017-02-21 11:30:00,100.15,99.2171,97.7135,98.1862,98.7105,0.524271,1.16772,100.049,95.378,100.16,93.15,2017-02-21 11:30:00
2017-02-21 12:30:00,100.3,99.2627,97.7438,98.2148,98.749,0.534197,1.1815,100.107,95.3808,100.253,93.3,2017-02-21 12:30:00


In [123]:
# make new dataframe starting with date and content
df = stock_df.copy() #'content_stopped',

# create a series with all hours covering entire twitter_df
time_index_hour_bins = pd.date_range(start = stock_df.index[0].floor('H'), end = stock_df.index[-1].ceil('H'), freq='H')#.to_period()

# bin the twitter_df.index by the new time_index_hour_bins
df['time_index_by_hour'] = pd.cut(stock_df['date'], time_index_hour_bins)

# extract JUST the hour_of_day as a feature
df['hour_of_day'] = df['date'].apply(lambda x: x.hour).apply(lambda x: dt.time(x))
df.drop('date',axis=1,inplace=True)
df.head()

,price,ma7,ma21,26ema,12ema,MACD,20sd,upper_band,lower_band,ema,momentum,time_index_by_hour,hour_of_day
date,,,,,,,,,,,,,
2017-02-20 15:30:00,99.64,99.080816,97.630272,98.104742,98.601714,0.496972,1.132580,99.895431,95.365113,99.639971,92.64,"(2017-02-20 15:00:00, 2017-02-20 16:00:00]",15:00:00
2017-02-21 09:30:00,99.88,99.121837,97.655102,98.128893,98.632705,0.503812,1.140074,99.935250,95.374954,99.799990,92.88,"(2017-02-21 09:00:00, 2017-02-21 10:00:00]",09:00:00
2017-02-21 10:30:00,100.37,99.172041,97.684830,98.159300,98.674795,0.515495,1.155433,99.995695,95.373965,100.179997,93.37,"(2017-02-21 10:00:00, 2017-02-21 11:00:00]",10:00:00
2017-02-21 11:30:00,100.15,99.217143,97.713469,98.186238,98.710509,0.524271,1.167723,100.048916,95.378023,100.159999,93.15,"(2017-02-21 11:00:00, 2017-02-21 12:00:00]",11:00:00
2017-02-21 12:30:00,100.30,99.262653,97.743810,98.214767,98.748965,0.534197,1.181503,100.106816,95.380803,100.253333,93.30,"(2017-02-21 12:00:00, 2017-02-21 13:00:00]",12:00:00


# LOADING IN TWITTER DATA

In [48]:
del(twitter_df)

In [49]:
try: twitter_df
except NameError: twitter_df = None
    
if twitter_df is None:
    print('loading twitter_df')
    twitter_df = pd.read_csv('data/trump_twitter_archive_df.csv', encoding='utf-8')
    twitter_df.drop('Unnamed: 0',axis=1,inplace=True)
    
    twitter_df['date']  = pd.to_datetime(twitter_df['date'])
    twitter_df['time_index'] = twitter_df['date'].copy()
    twitter_df.set_index('time_index',inplace=True,drop=True)

    # Fill in missing values before merging with stock data
    twitter_df.fillna('', inplace=True)
    twitter_df.sort_index(ascending=True, inplace=True)

    # Check header and daterange of index
    display(twitter_df.head(2))
    twitter_df.index[[0,-1]]
else:
    print('twitter_df already exists.')

loading twitter_df


,source,content,date,retweet_count,favorite_count,is_retweet,id_str,case_ratio,content_raw,has_RT,...,content_stopped,tokens_stopped,hashtag_strings,mention_strings,sentiment_scores,compound_score,sentiment_class,neg,neu,pos
time_index,,,,,,,,,,,,,,,,,,,,,
2017-01-20 12:31:53,Twitter for Android,It all begins today! I will see you at 11:00 A...,2017-01-20 12:31:53,70523,268372,False,822421390125043713,0.33036,It all begins today! I will see you at 11:00 A...,False,...,begins today see swearing movement continues w...,"['begins', 'today', 'see', 'swearing', 'moveme...",,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,0.0,1.000,0.000
2017-01-20 17:51:25,Twitter for iPhone,Today we are not merely transferring power fro...,2017-01-20 17:51:25,20125,109640,False,822501803615014918,0.01481,Today we are not merely transferring power fro...,False,...,today merely transferring power one administra...,"['today', 'merely', 'transferring', 'power', '...",,,"{'neg': 0.0, 'neu': 0.919, 'pos': 0.081, 'comp...",0.2144,pos,0.0,0.919,0.081


# How to merge stock-market-business hours tweets?
- use `custom_BH_freq` to create a new column that attempts to re-cast the index as this frequency.
    - separate the index off first to another column/dataframe.
    - create a second column that rounds/floors/ceils the timestamp values to the correct business hour
    - **NOTE**: -filling? revisit

#### USING ROLLBACK - PANDAS GUIDE


- from the https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

```python
In [148]: ts = pd.Timestamp('2018-01-06 00:00:00')

In [149]: ts.day_name()
Out[149]: 'Saturday'

# BusinessHour's valid offset dates are Monday through Friday
In [150]: offset = pd.offsets.BusinessHour(start='09:00')

# Bring the date to the closest offset date (Monday)
In [151]: offset.rollforward(ts)
Out[151]: Timestamp('2018-01-08 09:00:00')

# Date is brought to the closest offset date first and then the hour is added
In [152]: ts + offset
Out[152]: Timestamp('2018-01-08 10:00:00')
```

In [12]:
# df = twitter_df[['content','tokens_stopped']]

In [13]:
# CBH_off = pd.offsets.CustomBusinessHour(start='9:30', end='4:30')
# CBH_off

<CustomBusinessHour: CBH=09:30-04:30>

# BOOKMARK 07/01

In [115]:
# display(twitter_df.head(2))
# display(twitter_df.info())

In [81]:
twitter_df.index

DatetimeIndex(['2017-01-20 12:31:53', '2017-01-20 17:51:25',
               '2017-01-20 17:51:58', '2017-01-20 17:52:45',
               '2017-01-20 17:53:17', '2017-01-20 17:54:00',
               '2017-01-20 17:54:36', '2017-01-20 17:55:44',
               '2017-01-20 17:58:24', '2017-01-20 18:00:43',
               ...
               '2019-06-19 14:18:11', '2019-06-19 14:18:12',
               '2019-06-19 14:20:04', '2019-06-19 15:09:18',
               '2019-06-19 19:48:02', '2019-06-19 19:48:04',
               '2019-06-19 20:29:24', '2019-06-19 20:39:28',
               '2019-06-19 23:01:59', '2019-06-20 00:12:31'],
              dtype='datetime64[ns]', name='time_index', length=8442, freq=None)

In [124]:
# make new dataframe starting with date and content
df_tweet = twitter_df[['date','content','tokens_stopped']].copy() #'content_stopped',

# create a series with all hours covering entire twitter_df
time_index_hour_bins = pd.date_range(start = twitter_df.index[0].floor('H'), end = twitter_df.index[-1].ceil('H'), freq='H')#.to_period()

# bin the twitter_df.index by the new time_index_hour_bins
df_tweet['time_index_by_hour'] = pd.cut(twitter_df['date'], time_index_hour_bins)

# extract JUST the hour_of_day as a feature
df_tweet['hour_of_day'] = df_tweet['date'].apply(lambda x: x.hour).apply(lambda x: dt.time(x))
df_tweet.drop('date',axis=1,inplace=True)
df_tweet.head()

,content,tokens_stopped,time_index_by_hour,hour_of_day
time_index,,,,
2017-01-20 12:31:53,It all begins today! I will see you at 11:00 A...,"['begins', 'today', 'see', 'swearing', 'moveme...","(2017-01-20 12:00:00, 2017-01-20 13:00:00]",12:00:00
2017-01-20 17:51:25,Today we are not merely transferring power fro...,"['today', 'merely', 'transferring', 'power', '...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00
2017-01-20 17:51:58,power from Washington D.C. and giving it back ...,"['power', 'washington', 'c', 'giving', 'back',...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00
2017-01-20 17:52:45,What truly matters is not which party controls...,"['truly', 'matters', 'party', 'controls', 'gov...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00
2017-01-20 17:53:17,January 20th 2017 will be remembered as the da...,"['january', 'th', 'remembered', 'day', 'people...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00


In [100]:
twitter_df.columns

Index(['source', 'content', 'date', 'retweet_count', 'favorite_count',
       'is_retweet', 'id_str', 'case_ratio', 'content_raw', 'has_RT',
       'starts_RT', 'content_starts_RT', 'content_min_clean',
       'content_hashtags', 'content_mentions', 'content_stopped',
       'tokens_stopped', 'hashtag_strings', 'mention_strings',
       'sentiment_scores', 'compound_score', 'sentiment_class', 'neg', 'neu',
       'pos', 'hour_bins'],
      dtype='object')

In [125]:
cols_to_add = ['retweet_count','favorite_count','case_ratio','compound_score','neg','neu','pos']
df_tweet = pd.concat([df_tweet, twitter_df[cols_to_add]], axis=1)
df_tweet.head()

,content,tokens_stopped,time_index_by_hour,hour_of_day,retweet_count,favorite_count,case_ratio,compound_score,neg,neu,pos
time_index,,,,,,,,,,,
2017-01-20 12:31:53,It all begins today! I will see you at 11:00 A...,"['begins', 'today', 'see', 'swearing', 'moveme...","(2017-01-20 12:00:00, 2017-01-20 13:00:00]",12:00:00,70523,268372,0.33036,0.0000,0.000,1.000,0.000
2017-01-20 17:51:25,Today we are not merely transferring power fro...,"['today', 'merely', 'transferring', 'power', '...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,20125,109640,0.01481,0.2144,0.000,0.919,0.081
2017-01-20 17:51:58,power from Washington D.C. and giving it back ...,"['power', 'washington', 'c', 'giving', 'back',...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,18362,91143,0.07778,0.3400,0.000,0.844,0.156
2017-01-20 17:52:45,What truly matters is not which party controls...,"['truly', 'matters', 'party', 'controls', 'gov...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,42790,180394,0.00855,0.0954,0.079,0.776,0.145
2017-01-20 17:53:17,January 20th 2017 will be remembered as the da...,"['january', 'th', 'remembered', 'day', 'people...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,60604,218555,0.01020,0.0000,0.000,1.000,0.000


# Merging Dataframes

In [127]:
display(df.head(), df_tweet.head())

,price,ma7,ma21,26ema,12ema,MACD,20sd,upper_band,lower_band,ema,momentum,time_index_by_hour,hour_of_day
date,,,,,,,,,,,,,
2017-02-20 15:30:00,99.64,99.080816,97.630272,98.104742,98.601714,0.496972,1.132580,99.895431,95.365113,99.639971,92.64,"(2017-02-20 15:00:00, 2017-02-20 16:00:00]",15:00:00
2017-02-21 09:30:00,99.88,99.121837,97.655102,98.128893,98.632705,0.503812,1.140074,99.935250,95.374954,99.799990,92.88,"(2017-02-21 09:00:00, 2017-02-21 10:00:00]",09:00:00
2017-02-21 10:30:00,100.37,99.172041,97.684830,98.159300,98.674795,0.515495,1.155433,99.995695,95.373965,100.179997,93.37,"(2017-02-21 10:00:00, 2017-02-21 11:00:00]",10:00:00
2017-02-21 11:30:00,100.15,99.217143,97.713469,98.186238,98.710509,0.524271,1.167723,100.048916,95.378023,100.159999,93.15,"(2017-02-21 11:00:00, 2017-02-21 12:00:00]",11:00:00
2017-02-21 12:30:00,100.30,99.262653,97.743810,98.214767,98.748965,0.534197,1.181503,100.106816,95.380803,100.253333,93.30,"(2017-02-21 12:00:00, 2017-02-21 13:00:00]",12:00:00


,content,tokens_stopped,time_index_by_hour,hour_of_day,retweet_count,favorite_count,case_ratio,compound_score,neg,neu,pos
time_index,,,,,,,,,,,
2017-01-20 12:31:53,It all begins today! I will see you at 11:00 A...,"['begins', 'today', 'see', 'swearing', 'moveme...","(2017-01-20 12:00:00, 2017-01-20 13:00:00]",12:00:00,70523,268372,0.33036,0.0000,0.000,1.000,0.000
2017-01-20 17:51:25,Today we are not merely transferring power fro...,"['today', 'merely', 'transferring', 'power', '...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,20125,109640,0.01481,0.2144,0.000,0.919,0.081
2017-01-20 17:51:58,power from Washington D.C. and giving it back ...,"['power', 'washington', 'c', 'giving', 'back',...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,18362,91143,0.07778,0.3400,0.000,0.844,0.156
2017-01-20 17:52:45,What truly matters is not which party controls...,"['truly', 'matters', 'party', 'controls', 'gov...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,42790,180394,0.00855,0.0954,0.079,0.776,0.145
2017-01-20 17:53:17,January 20th 2017 will be remembered as the da...,"['january', 'th', 'remembered', 'day', 'people...","(2017-01-20 17:00:00, 2017-01-20 18:00:00]",17:00:00,60604,218555,0.01020,0.0000,0.000,1.000,0.000


In [165]:
stock_timebins[0].as_type('str')

AttributeError: 'pandas._libs.interval.Interval' object has no attribute 'as_type'

In [163]:
[x.index for x in stock_timebins if x.values in tweet_timebins.values]

AttributeError: 'pandas._libs.interval.Interval' object has no attribute 'values'

In [162]:
stock_timebins = df['time_index_by_hour']
tweet_timebins = df_tweet['time_index_by_hour']


print('stocks:')
print(type(stock_timebins))
print(stock_timebins[[0,-1]])
print('\n\ntweets:')
print(type(tweet_timebins))
print(tweet_timebins[[0,-1]])

stocks:
<class 'pandas.core.series.Series'>
date
2017-02-20 15:30:00    (2017-02-20 15:00:00, 2017-02-20 16:00:00]
2019-06-21 15:30:00    (2019-06-21 15:00:00, 2019-06-21 16:00:00]
Name: time_index_by_hour, dtype: category
Categories (20425, interval[datetime64[ns]]): [(2017-02-20 15:00:00, 2017-02-20 16:00:00] < (2017-02-20 16:00:00, 2017-02-20 17:00:00] < (2017-02-20 17:00:00, 2017-02-20 18:00:00] < (2017-02-20 18:00:00, 2017-02-20 19:00:00] ... (2019-06-21 12:00:00, 2019-06-21 13:00:00] < (2019-06-21 13:00:00, 2019-06-21 14:00:00] < (2019-06-21 14:00:00, 2019-06-21 15:00:00] < (2019-06-21 15:00:00, 2019-06-21 16:00:00]]


tweets:
<class 'pandas.core.series.Series'>
time_index
2017-01-20 12:31:53    (2017-01-20 12:00:00, 2017-01-20 13:00:00]
2019-06-20 00:12:31             (2019-06-20, 2019-06-20 01:00:00]
Name: time_index_by_hour, dtype: category
Categories (21133, interval[datetime64[ns]]): [(2017-01-20 12:00:00, 2017-01-20 13:00:00] < (2017-01-20 13:00:00, 2017-01-20 14:00:00] < (

In [160]:
stock_timebins[5] in tweet_timebins

False

In [140]:
df_tweet.groupby('time_index_by_hour'). groups

{Interval('2017-01-20 12:00:00', '2017-01-20 13:00:00', closed='right'): DatetimeIndex(['2017-01-20 12:31:53'], dtype='datetime64[ns]', name='time_index', freq=None),
 Interval('2017-01-20 13:00:00', '2017-01-20 14:00:00', closed='right'): DatetimeIndex([], dtype='datetime64[ns]', name='time_index', freq=None),
 Interval('2017-01-20 14:00:00', '2017-01-20 15:00:00', closed='right'): DatetimeIndex([], dtype='datetime64[ns]', name='time_index', freq=None),
 Interval('2017-01-20 15:00:00', '2017-01-20 16:00:00', closed='right'): DatetimeIndex([], dtype='datetime64[ns]', name='time_index', freq=None),
 Interval('2017-01-20 16:00:00', '2017-01-20 17:00:00', closed='right'): DatetimeIndex([], dtype='datetime64[ns]', name='time_index', freq=None),
 Interval('2017-01-20 17:00:00', '2017-01-20 18:00:00', closed='right'): DatetimeIndex(['2017-01-20 17:51:25', '2017-01-20 17:51:58',
                '2017-01-20 17:52:45', '2017-01-20 17:53:17',
                '2017-01-20 17:54:00', '2017-01-20 17

In [105]:
from pandas_profiling import ProfileReport
ProfileReport(df)

Number of variables,12
Number of observations,8442
Total Missing (%),0.0%
Total size in memory,1.0 MiB
Average record size in memory,130.1 B
Numeric,7
Categorical,4
Boolean,0
Date,1
Text (Unique),0
Rejected,0


In [ ]:
# verify this code
# # thoughts on how to bin twitter data

# df=pd.DataFrame()
# df['time_index'] = test_twitter_df.index.to_list()
# df.reset_index(inplace=True)

# content = test_twitter_df['content_stopped'].values
# df['content_stopped'] = content#twitter_df['content_stopped']

# df.reset_index(inplace=True)
# df.drop(['level_0','index'], inplace=True, axis=1)
# df.head()
# print(df.dtypes)

# # df.reindex('time_index')
# # df=df.asfreq('S')
# # df.head()

# NLP NEURAL NETWORK _ MOD 4


In [106]:
import mod4functions_JMI as jmi

import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import random
mpl.rcParams['figure.figsize']=(10,8)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Keras preprocessing
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

# Keras neural network basics
from keras import models
from keras import layers
from keras import optimizers

Using TensorFlow backend.


In [107]:
df_tokenize=df

In [108]:
# text_data = df_tokenize['content_min_clean']
text_data = df_tokenize['content']
from gensim.models import Word2Vec
vector_size = 300

wv_keras = Word2Vec(text_data, size=vector_size, window=10, min_count=1, workers=4)
wv_keras.train(text_data,total_examples=wv_keras.corpus_count, epochs=10)

wv = wv_keras.wv
vocab_size = len(wv_keras.wv.vocab)
print(f'There are {vocab_size} words in the word2vec vocabulary, with a vector size {vector_size}.')

There are 437 words in the word2vec vocabulary, with a vector size 300.


In [109]:
# save the vectors in a new matrix
word_model = wv_keras
vector_size = word_model.wv.vectors[1].shape[0]

embedding_matrix = np.zeros((len(word_model.wv.vocab) + 1, vector_size))
for i, vec in enumerate(word_model.wv.vectors):
  embedding_matrix[i] = vec
embedding_matrix.shape

(438, 300)

In [110]:
# Get list of texts to be converted to sequences
# sentences_train =text_data # df_tokenize['tokens'].values
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=len(wv.vocab))
tokenizer.fit_on_texts(list(text_data)) #tokenizer.fit_on_texts(text_data)

word_index = tokenizer.index_word
reverse_index = {v:k for k,v in word_index.items()}

In [111]:
# return integer-encoded sentences
from keras.preprocessing import text, sequence
X = tokenizer.texts_to_sequences(text_data)
X = sequence.pad_sequences(X)

y = df_tokenize['troll_tweet'].values
# reverse_index
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, y)#, test_size=0.1, shuffle=False)

KeyError: 'troll_tweet'

In [ ]:
from keras import models, layers, optimizers, regularizers

model2 = models.Sequential()
# embedding_layer = wv.get_keras_embedding(train_embeddings=False)

model2.add(layers.Embedding(len(wv_keras.wv.vocab)+1,
                             vector_size,input_length=X_train.shape[1],
                             weights=[embedding_matrix],trainable=False)) 
          
model2.add(layers.LSTM(300, return_sequences=False))#, kernel_regularizer=regularizers.l2(.01)))
# model1B.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
model2.summary()

In [ ]:
clock = bs.Clock()
clock.tic('starting keras .fit')
num_epochs = 5
history = model2.fit(X_train, y_train, epochs=num_epochs, verbose=True, validation_data=(X_val, y_val), batch_size=500)

clock.toc(f'completed {num_epochs} epochs')

loss, accuracy = model2.evaluate(X_train, y_train, verbose=True)
print(f'Training Accuracy:{accuracy}')

loss, accuracy = model2.evaluate(X_test, y_test, verbose=True)
print(f'Testing Accuracy:{accuracy}')

jmi.plot_keras_history(history)